In [33]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import precision_score, recall_score


In [81]:
cancelation_df = pd.read_csv('../../raw_data/latlong_ohe.csv')

In [82]:
cancelation_df.shape

(714851, 25)

In [83]:
cancelation_df = cancelation_df.sample(50000)

In [84]:
cancelation_df.head(5)

,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DESTINATION_LONGITUDE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
184220,7,26,7,75.0,327,1250,0,-71.42822,41.72400,-76.66820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
472050,9,20,7,350.0,2552,2155,0,-112.00806,33.43417,-149.99619,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45584,5,11,1,255.0,1728,1440,0,-87.75242,41.78598,-117.18966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
91867,9,9,3,174.0,1119,2219,0,-80.15275,26.07258,-97.03720,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
669583,3,6,5,187.0,946,1017,1,-74.16866,40.69250,-89.97667,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
#Borrar NaN
cancelation_df = cancelation_df.dropna()

In [86]:
X = cancelation_df.drop(columns = ['CANCELLED'])
y = cancelation_df['CANCELLED']

In [87]:
X.shape, y.shape

((49991, 24), (49991,))

In [88]:
y.value_counts()

CANCELLED
0    43841
1     6150
Name: count, dtype: int64

In [89]:
# STEP 0 - Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [90]:
#Standard Scaler
std_scaler = StandardScaler()
X_train_sc = std_scaler.fit_transform(X_train)
X_test_sc=std_scaler.transform(X_test)

In [91]:
# Instanciate the model
knn_model = KNeighborsClassifier(n_neighbors=8)

knn_model.fit(X_train_sc, y_train)

# Train the model on the scaled Training data
cv_results = cross_validate(knn_model, X_train_sc, y_train, cv=5, scoring=['precision'])


In [92]:
knn_score = cv_results['test_precision'].mean()
knn_score

0.6218834259182934

In [93]:
y_pred= knn_model.predict(X_test_sc)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [95]:
#Predecir para vuelos cancelados 1: BASELINE
y_predict_baseline = np.ones(len(y_train))

precision_baseline = precision_score(y_train, y_predict_baseline)

precision_baseline

0.12228159917697826

In [96]:
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()

# n_neighbors
k_range = list(range(1,30))

# weights
weigh_options = ["uniform", "distance"]

# P
p = [1,2] 

param_grid = dict(n_neighbors = k_range, weights = weigh_options , p = p) 

# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=5, scoring='precision', verbose=1, n_jobs=-1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train_sc, y_train)

Fitting 5 folds for each of 116 candidates, totalling 580 fits


In [97]:
print(grid_search.best_params_)

{'n_neighbors': 25, 'p': 1, 'weights': 'distance'}


In [98]:
print(grid.best_score_)

0.7266247528728641


In [101]:
# Instanciate the model
knn_model = KNeighborsClassifier(n_neighbors=25, p=1, weights='distance')

knn_model.fit(X_train_sc, y_train)

# Train the model on the scaled Training data
cv_results = cross_validate(knn_model, X_train_sc, y_train, cv=5, scoring=['precision'])

In [102]:
knn_score = cv_results['test_precision'].mean()
knn_score

0.7266247528728641